# Import
原本数据中的Lowess做的不是很好，这里矫正一下，并且加入降采样

In [1]:
import os
import re
import gc
import sys 
from datetime import datetime
from loguru import logger



import torch
import numpy as np
from scipy.linalg import eig, svd, solve
from statsmodels.nonparametric.smoothers_lowess import lowess


import matplotlib.pyplot as plt
%matplotlib qt




# Loewner

## Definition

In [2]:

def Loewner_Framework(f, Z, REALFLAG = True):
    '''==================================================
        Construct Loewner Pencel
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        ==================================================
    '''
    _n = len(f)
    s = 2j * np.pi * f

    # Ensuring the input have an even number of elements 
    # for constructing the model having real entries
    if REALFLAG:
        if _n % 2 != 0:
            _n = _n - 1

    # Left & Right Data for Loewner Framework
    s_left  = s[:_n:2]
    H_left  = Z[:_n:2]
    s_right = s[1:_n:2] 
    H_right = Z[1:_n:2]

    # Construct complex conjugate values for ensuring model having real entries
    if REALFLAG:
        s_left  = np.stack([s_left, s_left.conj()], axis=1).flatten()
        H_left  = np.stack([H_left, H_left.conj()], axis=1).flatten()
        s_right = np.stack([s_right, s_right.conj()], axis=1).flatten()
        H_right = np.stack([H_right, H_right.conj()], axis=1).flatten()

    # Constructing the Loewner Matrix & Shifted Loewner Matrix
    # L   = (H_left[:,None] - H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    # Ls  = (s_left[:,None] * H_left[:,None] - s_right[None,:] * H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    L   = (H_left[None,:] - H_right[:,None]) / (s_left[None,:] - s_right[:,None])
    Ls  = (s_left[None,:] * H_left[None,:] - s_right[:,None] * H_right[:,None]) / (s_left[None,:] - s_right[:,None])

    # Transforming the conplex L & Ls to obtain matrices with real entries
    if REALFLAG:
        _J_diag = np.eye(_n//2)
        _J  = (1/np.sqrt(2)) * np.array([[1, 1j], [1, -1j]])
        _J  = np.kron(_J_diag, _J)

        L       = (_J.conj().T @ L @ _J).real
        Ls      = (_J.conj().T @ Ls @ _J).real
        H_left  = ((_J.T @ H_left).T).real
        H_right = (_J.conj().T @ H_right).real

        
    return L, Ls, H_left, H_right

def state_space_model(L, Ls, H_left, H_right):
    '''==================================================
        Construct state space model from Loewner Pencel
        Parameter: 
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        Returen:
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        ==================================================
    '''
    # rank of the Loewner Pencel
    _rank = np.linalg.matrix_rank(np.concatenate((L, Ls), axis=0))
    Y_L, svd_L, X_L = svd(L, full_matrices=False, lapack_driver='gesvd')
    X_L = X_L.T
    
    # Reduced state space model interpolating the data
    Yk = Y_L[:, :_rank]
    Xk = X_L[:, :_rank]

    Ek = -Yk.T@L@Xk
    Ak = -Yk.T@Ls@Xk
    Bk = Yk.T@H_right
    Ck = H_left.T@Xk

    return Ek, Ak, Bk, Ck

def DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG = True, real_th = 1e3):
    '''==================================================
        Transform state space model to DRT model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        Returen:
            tau_i   tau_i from RC pair in DRT
            R_i:    R_i from RC pair in DRT
            C_i:    C_i from RC pair in DRT
        ==================================================
    '''
    # Solve Av= λEv & wT A= λ wT E & Res = CvwB/wEv, wEv =  δ
    _pol, _U = eig(Ak, Ek)     # 
    wB = solve(_U, solve(Ek,Bk))
    Cv = Ck @ _U
    _res = Cv * wB

    # Calculate R_i & tau_i
    tau_i   = (-1/_pol) 
    R_i     = (-_res / _pol)
    C_i     = (1/_res)

    if REALFLAG:
        real_ratio = np.where(np.abs(tau_i.imag) == 0, np.inf, np.abs(tau_i.real / (tau_i.imag+1e-20)))
        tau_i = np.abs(tau_i[real_ratio > real_th])
        R_i = np.abs(R_i[real_ratio > real_th])
        C_i = np.abs(C_i[real_ratio > real_th])


    return tau_i, R_i, C_i

def DRT_Reconstruction_SSM(Ek, Ak, Bk, Ck, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            R_i:    R_i from RC pair in DRT
            tau_i   tau_i from RC pair in DRT
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f
    H = np.array([Ck @ solve(si * Ek - Ak, Bk) for si in s])
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ


def DRT_Reconstruction_DRT(R_i, tau_i, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f  # Broadcasting tau_i to match f
    _RC = R_i[None, :] / (1+s[:,None] * tau_i[None,:])
    H = np.sum(_RC, axis=1)

    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ

def DRT_singularity_analysis(f, Z, REALFLAG = True):
    '''==================================================
        DRT Singularity Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Y, svd_L, X = svd(np.concatenate([L, Ls]), full_matrices=False)

    return svd_L

def DRT_Calculation(f, Z, REALFLAG = True):
    '''==================================================
        DRT Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
            H:          reconstructed impedance values
            res_ReZ:    relative error of real part of impedance values
            res_ImZ:    relative error of imaginary part of impedance values
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Ek, Ak, Bk, Ck = state_space_model(L, Ls, H_left, H_right)
    tau_i, R_i, C_i = DRT_Transform(Ek, Ak, Bk, Ck)

    return tau_i, R_i, C_i

def DRT_Bootstrap(f, Z, REALFLAG = True, n_batch = 100, n_point = 100, idx_base=1000):
    '''==================================================
        DRT Analysis with Bootstrap for one channel
        Parameter: 
            f:          frequency values with full range (5000)
            Z:          impedance values (H = Z)
            n_batch:    number of bootstrap samples
            n_point:    number of points for each sample
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            DRTdata:    list of tuples (tau_i, R_i, C_i) for each sample
            EISdata:    list of tuples (f, Z.real, Z.imag) for each sample
        ==================================================
    '''
    DRTdata = []
    EISdata = []
    for _ in range(n_batch):

        
        indices = np.arange(idx_base, f.shape[0])
        bins = np.array_split(indices, n_point)
        sampled_indices = [np.random.choice(bin, 1)[0] for bin in bins]
        
        f_sampled = f[sampled_indices]
        Z_sampled = Z[sampled_indices]

        _tau_i, _R_i, _C_i = DRT_Calculation(f_sampled, Z_sampled, REALFLAG)
        DRTdata.append((np.array([_tau_i, _R_i, _C_i])))
        EISdata.append((np.array([f_sampled, Z_sampled.real, Z_sampled.imag])))

    return DRTdata, EISdata

def DRT_Loess(DRTdata):
    '''==================================================
        DRT Analysis with Loess
        Parameter: 
            DRTdata:    list of tuples (tau_i, R_i, C_i) for each sample
        Returen:
            DRTdata_Loess:    Loess smoothed DRT data
        ==================================================
    '''

    # _tau_i  = np.concatenate([i[0,1:-1] for i in DRTdata])
    # _R_i    = np.concatenate([i[1,1:-1] for i in DRTdata])
    # _C_i    = np.concatenate([i[2,1:-1] for i in DRTdata])
    _tau_i_org  = np.concatenate([i[0,:] for i in DRTdata])
    _R_i_org    = np.concatenate([i[1,:] for i in DRTdata])
    _C_i_org    = np.concatenate([i[2,:] for i in DRTdata])

    _mask = (_tau_i_org>1e-7) & (_tau_i_org<1e-2)
    _tau_i  = _tau_i_org[_mask]
    _R_i    = _R_i_org[_mask]
    _C_i    = _C_i_org[_mask]

    x_log = np.log(_tau_i)
    y_log = np.log(_R_i)-np.log(_C_i)
    
    y_log_smooth = lowess(y_log, x_log, frac=0.1, it=3, return_sorted=False)


    _tau_i_org[_mask]  = _tau_i
    _R_i_org[_mask]    = np.exp((x_log + y_log_smooth)/2)
    _C_i_org[_mask]    = _tau_i / _R_i_org[_mask]


    R_loess         = _R_i_org
    tau_loess       = _tau_i_org
    C_loess         = _C_i_org

    DRTdata_Loess   =  np.array([tau_loess, R_loess, C_loess])

    return DRTdata_Loess

# def DRT_Analysis_Batch(chData, REALFLAG = True):
#     '''==================================================
#         DRT Analysis for Batch Data - without eis_seq filtering
#         Parameter: 
#             chData:     list of tuples (f, Z) for each channel
#             REALFLAG:   boolean flag to indicate if the model should have real entries
#         Returen:
#             results:    list of tuples (R_i, C_i, tau_i, H, res_ReZ, res_ImZ) for each channel
#         ==================================================
#     '''
#     DRTdata_list = []
#     EISdata_list = []
#     Loess_list = []
#     f = chData[0,0,:]
#     for i in range(chData.shape[0]):
#         _Z = chData[i,1,:] + 1j*chData[i,2,:]
#         DRTdata, EISdata = DRT_Bootstrap(f, _Z, REALFLAG=REALFLAG, n_batch=n_batch, n_point=n_point, idx_base=idx_base)
#         DRTdata_Loess = DRT_Loess(DRTdata)

#         DRTdata_list.append(DRTdata)
#         EISdata_list.append(EISdata)
#         Loess_list.append(DRTdata_Loess)

#     return DRTdata_list, EISdata_list, Loess_list


def DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq):
    
    axis = [0] * 6
    axis[0] = fig.add_subplot(2,3,1)    # Nyquist Plot
    axis[1] = fig.add_subplot(2,3,2)    # Bode Plot (Magnitude)
    axis[2] = fig.add_subplot(2,3,3)    # Bode Plot (Phase)
    axis[3] = fig.add_subplot(2,3,4)    # DRT (RC)
    axis[4] = fig.add_subplot(2,3,5)    # DRT (Rτ)
    axis[5] = fig.add_subplot(2,3,6)    # DRT (Fixed)



    _s       = 2
    _alpha   = 0.7

    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(EISdata_list)):
        if i in eis_seq:
            ch_eis      = EISdata_list[i][0]
            ch_drt      = DRTdata_list[i]
            ch_loess    = Loess_list[i]

            # ch_R    = np.array([i[1:,0] for i in ch_drt])
            # ch_C    = np.array([i[1:,-1] for i in ch_drt])
            ch_R    = np.concatenate([i[1,:] for i in ch_drt])
            ch_C    = np.concatenate([i[2,:] for i in ch_drt])

            _color  = cmap(i/len(EISdata_list))

            axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
            axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)

            axis[3].scatter(ch_R, ch_C, s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[4].scatter(ch_loess[0,:], ch_loess[1,:]/ch_loess[2,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            # axis[4].scatter(ch_loess[0,:], ch_loess[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            # axis[4].scatter(ch_loess[0,:], ch_loess[2,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[5].scatter(ch_loess[0,:], (ch_loess[1,:]/ch_loess[2,:])/(((1e6/ch_loess[0,:])) + ((ch_loess[0,:]/5e-15))), s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            # axis[5].scatter(ch_loess[0,:], (ch_loess[1,:]/ch_loess[2,:])/(((1e3/ch_loess[0,:])) + ((ch_loess[0,:]/1e-7))), s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')



    axis[0].set_aspect('equal', adjustable='datalim')
    axis[3].set_xscale('log')
    axis[3].set_yscale('log')
    axis[4].set_xscale('log')
    axis[4].set_yscale('log')
    axis[5].set_xscale('log')
    axis[5].set_yscale('log')


    # return text_axis


## Data Loader

In [3]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list

In [4]:
def setup_logger(log_dir="./LOG", log_filename="file.log", file_level="WARNING", console_level="WARNING"):
    # 创建目录
    os.makedirs(log_dir, exist_ok=True)
    log_fd = os.path.join(log_dir, log_filename)

    logger.remove()
    # 如果已有日志文件，重命名添加时间戳
    if os.path.exists(log_fd):
        name, ext = os.path.splitext(log_filename)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archived_name = f"{name}_{timestamp}{ext}"
        archived_path = os.path.join(log_dir, archived_name)
        os.rename(log_fd, archived_path)

    # 添加终端输出
    logger.add(sys.stdout, level=console_level, enqueue=True)

    # 添加文件输出
    logger.add(log_fd, level=file_level, encoding="utf-8", enqueue=True)

    return logger

In [ ]:
if True:
    setup_logger(log_dir="D:\Baihm\EISNN\LOG\DRTLoe_Process_Log")

# logger.remove()
# logger.add(sys.stdout, level="WARNING")
# logger.add("./LOG/file.log", rotation="10 MB", level="INFO")

2025-07-29 01:14:42.460 | WARNING  | __main__:<module>:33 - ELE[1/187]: 	02027452 - D:/Baihm/EISNN/Archive_New/2025\1月\02027452_归档
2025-07-29 01:14:42.460 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[000] Begin
2025-07-29 01:14:43.631 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[001] Begin
2025-07-29 01:14:44.642 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[002] Begin
2025-07-29 01:14:45.705 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[003] Begin
2025-07-29 01:14:46.753 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[004] Begin
2025-07-29 01:14:47.972 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[007] Begin
2025-07-29 01:14:48.870 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[008] Begin
2025-07-29 01:14:49.897 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[009] Begin
2025-07-29 01:14:50.981 | WARNING  | __main__:<module>:49 - ELE[02027452][1/187] - ch[010] Be

In [6]:

# rootPath = "D:/Baihm/EISNN/Archive/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"

rootPath = "D:/Baihm/EISNN/Archive_New/"
ele_list = SearchELE(rootPath)
DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Invivo/"
# ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver02"))
# DATASET_SUFFIX = "Outlier_Ver04"


n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

In [ ]:
# freq_list = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)
# freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
# freq_list = freq_list[1:]

DRT_SUFFIX = f"{DATASET_SUFFIX}_DRTLoe_Ver02"
SAVE_FLAG = False
FORCE_FLAG = False

In [8]:
RUN_FLAG = True
if RUN_FLAG:

    REALFLAG = True
    

    for i in range(n_ele):
    # for i in range(0,1):
        # logger.info(f"ELE Begin: {ele_list[i][0]}")
        fd_pt = os.path.join(ele_list[i][0], DRT_SUFFIX, f"{ele_list[i][1]}_{DRT_SUFFIX}.pt")
        if not os.path.exists(fd_pt):
            logger.warning(f"{fd_pt} does not exist")
            continue
        
        data_pt = torch.load(fd_pt)
        _meta_group = data_pt["meta_group"]
        _data_group = data_pt["data_group"]

        _process_debug_cnt = _meta_group.get('ProcessCount', 0)

        if _process_debug_cnt >= 2:
            logger.warning(f"ELE[{i+1}/{n_ele}]: {ele_list[i][1]} - ProcessCount {_meta_group['ProcessCount']} >= 2, skip")
            continue

        _meta_group['ProcessCount'] = 2

        ele_id  = _meta_group["ele_id"]
        elePath = _meta_group["elePath"]
        n_ch = _meta_group["n_ch"]      
        x_day_full = _meta_group["TimeSpan"]


        logger.warning(f"ELE[{i+1}/{n_ele}]: \t{ele_id} - {elePath}")


        # Storage path
        save_dir = f"{elePath}/{DRT_SUFFIX}/"
        pt_file_name = f"{ele_id}_{DRT_SUFFIX}.pt"
        # os.makedirs(save_dir, exist_ok=True)
        # if os.path.exists(os.path.join(save_dir, pt_file_name)):
        #     logger.warning(f"FileAlreadyExistsWarning: {ele_id} - {pt_file_name} already exists.")
        #     if SAVE_FLAG and not FORCE_FLAG:
        #         continue


        for j in _data_group['Channels']:
            try:
            # if 1:
                logger.warning(f"ELE[{ele_id}][{i+1}/{n_ele}] - ch[{j:03d}] Begin") 
                channel_group_raw = _data_group[j]


                chData          =   channel_group_raw['chData']         
                eis_seq         =   channel_group_raw['eis_seq']        
                # eis_cluster     =   channel_group_raw['eis_cluster']    
                # eis_anomaly     =   channel_group_raw['eis_anomaly']    
                # leaf_anomaly    =   channel_group_raw['leaf_anomaly']   
                # seq_weird       =   channel_group_raw['seq_weird']      
                # seq_open        =   channel_group_raw['seq_open']       
                # seq_short       =   channel_group_raw['seq_short']      




                if chData.shape[2] != 5000:
                    logger.error(f"ELE[{ele_id}] - ch[{j}] with less than 5000 samples")
                    break
                

                DRTdata_list    = channel_group_raw['DRTlist']
                EISdata_list    = channel_group_raw['EISlist']
                Loess_list      = channel_group_raw['Loesslist']

                # Run DRT Analysis
                f = chData[0,0,:]
                for k in range(chData.shape[0]):
                    if k in eis_seq:
                        DRTdata_Loess = DRT_Loess(DRTdata_list[k])
                        Loess_list[k] = DRTdata_Loess


                # Plot DRT Analysis Results
                fig = plt.figure(figsize=(16, 9), constrained_layout=True)
                fig.suptitle(f"ELE[{ele_id}] - ch[{j:03d}] LFDRT Analysis", fontsize=16, fontweight='bold')
                DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq)
                

                # Save Fig
                fig_name = f"DRT_{ele_id}_ch{j:03d}.png"
                
                os.makedirs(save_dir, exist_ok=True) 
                path = os.path.join(save_dir, fig_name)

                fig.savefig(path)
                plt.close('all') 

                # Data Saving
                channel_group_DRT = channel_group_raw
                channel_group_DRT['Loesslist']  = Loess_list

                _data_group[j] = channel_group_DRT
                logger.info(f"ELE[{ele_id}] - ch[{j:03d}] Finished")
                
                # break

            except Exception as e:
                logger.error(f"ELE[{ele_id}] - ch[{j:03d}] Run with error: {e}")
                continue

        
        pt_store = {}
        pt_store["meta_group"] = _meta_group
        pt_store["data_group"] = _data_group
        if SAVE_FLAG:
            tmp_path = "tmp.pt"
            torch.save(pt_store, tmp_path)
            os.replace(tmp_path, os.path.join(save_dir, pt_file_name)) 

        del data_pt, _meta_group, _data_group, pt_store
        gc.collect()


C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_5552\419198110.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_pt = torch.load(fd_pt)
C:\Users\Baihm\AppData\Local\Te

In [9]:
# plt.figure()
# fig = plt.gca()

# ch_loess    = Loess_list[0]
# ch_drt    = DRTdata_list[0]


# ch_R    = np.concatenate([i[1,:] for i in ch_drt])
# ch_C    = np.concatenate([i[2,:] for i in ch_drt])

# # fig.scatter(ch_loess[1,:],ch_loess[2,:])
# fig.scatter(ch_R,ch_C)
# # fig.scatter(ch_loess[0,:], (ch_loess[1,:]/ch_loess[2,:]))
# # fig.scatter(ch_loess[0,:], (((1e6/ch_loess[0,:])) + ((ch_loess[0,:]/1e-14))))
# # fig.scatter(ch_loess[0,:], (ch_loess[1,:]/ch_loess[2,:])/(((1e6/ch_loess[0,:])) + ((ch_loess[0,:]/1e-14))))
# fig.set_xscale('log')
# fig.set_yscale('log')



## Viewer

In [10]:
VIEWER_FLAG = False
if VIEWER_FLAG:
    # ele_id = "02067447"
    # ch_id = 16    # Short with two phased
    # ch_id = 68    # Short Type II

    
    ele_id = "06017758"
    ch_id = 10    # Perfect

    
    # ele_id = "01037160"
    # ch_id = 0    # Short type I

    
    # ele_id = "11057712"
    # # ch_id = 26      # Normal to Short
    # ch_id = 107      # Normal to Short
    
    fd_pt = f"D:/Baihm/EISNN/Archive/{ele_id}_归档\\Outlier_Ver04_DRTLoe_Ver02\\{ele_id}_Outlier_Ver04_DRTLoe_Ver02.pt"

    # Read out
    data_pt = torch.load(fd_pt)
    _data_group = data_pt["data_group"]
    eis_seq      =_data_group[ch_id]['eis_seq']

    DRTdata_list = _data_group[ch_id]['DRTlist']
    EISdata_list = _data_group[ch_id]['EISlist']
    Loess_list = _data_group[ch_id]['Loesslist']
    
    # Plot
    fig = plt.figure(figsize=(16, 9), constrained_layout=True)
    DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq)
    # text_axis = DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq)
    # DRT_Plot_Batch_3D(fig, DRTdata[:2], chData[:2,:,:])



In [11]:
print(len(EISdata_list))
print(len(DRTdata_list))

5
5
